In [2]:
import sys
sys.path.append('../')
import pandas as pd
import joblib
from helper_files.modelling_functions import get_data, process_data, run_optuna_trials, evaluate_models, get_data_config, get_best_model
from helper_files.modelling_configs import *
from helper_files.etv_model import ETVModel

/Users/bradeneberhard/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
data_path = '../data/processed/data_splits_1003.jblb'
train_df, test_df_time, test_df_random, test_df_thrower = get_data(data_path)
test_dfs = [test_df_random, test_df_time, test_df_thrower]
storage = 'sqlite:///model_tuning.db'
train_df = train_df[train_df['throw_distance'] >= 1]
MIRROR = True
NOISE_FACTOR = 0.03
SMOTE = False

In [3]:
# features = thrower_context
# target = 'outcome' 
# data_config = get_data_config(features, target, train_df, test_dfs, False)
# best_models, best_params = run_optuna_trials(data_config, model_config, suffix='fv_base_augment_all', delete_models=True, storage=storage)
# metrics_df = evaluate_models(best_models, data_config)
# metrics_df

In [9]:
fv_features = thrower_context + game_context
target = 'point_outcome'  
data_config = get_data_config(fv_features, target, train_df, test_dfs, mirror=MIRROR, noise_factor=NOISE_FACTOR, smote=SMOTE)
best_models, best_params = run_optuna_trials(data_config, model_config, suffix='fv', delete_models=True, storage=storage)
metrics_df = evaluate_models(best_models, data_config)
metrics_df

[I 2024-11-04 16:43:08,135] A new study created in RDB with name: logreg_fv
[I 2024-11-04 16:43:16,172] Trial 0 finished with value: 0.7135911291230739 and parameters: {}. Best is trial 0 with value: 0.7135911291230739.
[I 2024-11-04 16:43:18,029] A new study created in RDB with name: xgb_fv
[I 2024-11-04 16:43:52,936] Trial 0 finished with value: 0.7323007840896073 and parameters: {'n_estimators': 177, 'learning_rate': 0.5946256479919667, 'max_depth': 10}. Best is trial 0 with value: 0.7323007840896073.
[I 2024-11-04 16:44:08,933] Trial 1 finished with value: 0.7101630942420705 and parameters: {'n_estimators': 105, 'learning_rate': 0.0029551943633314404, 'max_depth': 7}. Best is trial 0 with value: 0.7323007840896073.
[I 2024-11-04 16:44:20,305] Trial 2 finished with value: 0.7101630942420705 and parameters: {'n_estimators': 63, 'learning_rate': 0.00024298916040752094, 'max_depth': 6}. Best is trial 0 with value: 0.7323007840896073.
[I 2024-11-04 16:44:30,706] Trial 3 finished with va

,Model,train_AUC,train_Accuracy,train_NPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV
0,logreg,0.641565,0.713602,0.565164,0.642111,0.712921,0.573733,0.638012,0.696611,0.551489,0.631650,0.732406,0.554517
1,xgb,0.720353,0.742344,0.746962,0.721588,0.743807,0.769908,0.716197,0.728441,0.751360,0.720027,0.762855,0.772642


In [4]:
fv_features = thrower_context + game_context
target = 'point_outcome'  
data_config = get_data_config(fv_features, target, train_df, test_dfs, mirror=MIRROR, noise_factor=NOISE_FACTOR, smote=SMOTE)
best_models, best_params = run_optuna_trials(data_config, model_config, suffix='fv', delete_models=True, storage=storage)
metrics_df = evaluate_models(best_models, data_config)
metrics_df

[I 2024-11-04 11:31:27,254] A new study created in RDB with name: logreg_fv
[I 2024-11-04 11:31:28,017] Trial 0 finished with value: 0.7136944299075578 and parameters: {}. Best is trial 0 with value: 0.7136944299075578.
[I 2024-11-04 11:31:28,251] A new study created in RDB with name: xgb_fv
[I 2024-11-04 11:31:31,117] Trial 0 finished with value: 0.7101630942450206 and parameters: {'n_estimators': 113, 'learning_rate': 0.002677610067924562, 'max_depth': 6}. Best is trial 0 with value: 0.7101630942450206.
[I 2024-11-04 11:31:32,134] Trial 1 finished with value: 0.7101630942450206 and parameters: {'n_estimators': 49, 'learning_rate': 0.001158361349102705, 'max_depth': 3}. Best is trial 0 with value: 0.7101630942450206.
[I 2024-11-04 11:31:33,680] Trial 2 finished with value: 0.728722568782082 and parameters: {'n_estimators': 75, 'learning_rate': 0.08384524806967683, 'max_depth': 5}. Best is trial 2 with value: 0.728722568782082.
[I 2024-11-04 11:31:35,348] Trial 3 finished with value: 0

,Model,train_AUC,train_Accuracy,train_NPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV
0,logreg,0.641755,0.713628,0.565229,0.642119,0.712921,0.572464,0.63797,0.696522,0.549663,0.631657,0.732519,0.556068
1,xgb,0.710977,0.740410,0.732653,0.711728,0.741287,0.745957,0.70525,0.724108,0.723305,0.709784,0.759821,0.751746


In [10]:
cp_features = thrower_context + receiver_context + throw_context + game_context
target = 'completion'
data_config = get_data_config(cp_features, target, train_df, test_dfs, mirror=MIRROR, noise_factor=NOISE_FACTOR, smote=SMOTE)
best_models, best_params = run_optuna_trials(data_config, model_config, suffix='cp', delete_models=True, storage=storage)
metrics_df = evaluate_models(best_models, data_config)
metrics_df

[I 2024-11-04 16:45:43,137] A new study created in RDB with name: logreg_cp
[I 2024-11-04 16:45:52,519] Trial 0 finished with value: 0.9375279952593267 and parameters: {}. Best is trial 0 with value: 0.9375279952593267.
[I 2024-11-04 16:45:54,282] A new study created in RDB with name: xgb_cp
[I 2024-11-04 16:46:01,900] Trial 0 finished with value: 0.941503786650326 and parameters: {'n_estimators': 50, 'learning_rate': 0.4960915734761351, 'max_depth': 6}. Best is trial 0 with value: 0.941503786650326.
[I 2024-11-04 16:46:28,633] Trial 1 finished with value: 0.9437694665456311 and parameters: {'n_estimators': 142, 'learning_rate': 0.11719589598826381, 'max_depth': 9}. Best is trial 1 with value: 0.9437694665456311.
[I 2024-11-04 16:46:46,052] Trial 2 finished with value: 0.938713788313947 and parameters: {'n_estimators': 93, 'learning_rate': 0.007533144870080817, 'max_depth': 8}. Best is trial 1 with value: 0.9437694665456311.
[I 2024-11-04 16:46:53,671] Trial 3 finished with value: 0.94

,Model,train_AUC,train_Accuracy,train_NPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV
0,logreg,0.793950,0.937529,0.448622,0.724159,0.932323,0.465116,0.689508,0.932921,0.432039,0.746070,0.935310,0.451977
1,xgb,0.897617,0.948896,0.900582,0.892229,0.943503,0.901316,0.885891,0.945564,0.886128,0.896914,0.948241,0.913747


In [6]:
cp_features = thrower_context + receiver_context + throw_context + game_context
target = 'completion'
data_config = get_data_config(cp_features, target, train_df, test_dfs, mirror=MIRROR, noise_factor=NOISE_FACTOR, smote=SMOTE)
best_models, best_params = run_optuna_trials(data_config, model_config, suffix='cp', delete_models=True, storage=storage)
metrics_df = evaluate_models(best_models, data_config)
metrics_df

[I 2024-11-04 11:31:52,029] A new study created in RDB with name: logreg_cp
[I 2024-11-04 11:31:53,528] Trial 0 finished with value: 0.9375713992762142 and parameters: {}. Best is trial 0 with value: 0.9375713992762142.
[I 2024-11-04 11:31:54,042] A new study created in RDB with name: xgb_cp
[I 2024-11-04 11:31:56,986] Trial 0 finished with value: 0.9386130913221434 and parameters: {'n_estimators': 109, 'learning_rate': 0.0008228061331417799, 'max_depth': 5}. Best is trial 0 with value: 0.9386130913221434.
[I 2024-11-04 11:31:59,106] Trial 1 finished with value: 0.9386130913221434 and parameters: {'n_estimators': 78, 'learning_rate': 0.005280267926068886, 'max_depth': 6}. Best is trial 0 with value: 0.9386130913221434.
[I 2024-11-04 11:32:02,368] Trial 2 finished with value: 0.9422138748050308 and parameters: {'n_estimators': 164, 'learning_rate': 0.1304441707180821, 'max_depth': 3}. Best is trial 2 with value: 0.9422138748050308.
[I 2024-11-04 11:32:09,838] Trial 3 finished with value

,Model,train_AUC,train_Accuracy,train_NPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV
0,logreg,0.795149,0.937578,0.450662,0.723918,0.932323,0.464924,0.688865,0.932891,0.430993,0.745490,0.935234,0.449153
1,xgb,0.826846,0.942651,0.719517,0.829265,0.936980,0.732857,0.829742,0.938226,0.715278,0.841154,0.940998,0.743697


In [7]:
# features = thrower_context + receiver_context + throw_context + game_context + player_context
# target = 'completion' 
# data_config = get_data_config(features, target, train_df, test_dfs, False)
# best_models, best_params = run_optuna_trials(data_config, model_config, suffix='cp_game_player', delete_models=True, storage=storage)
# metrics_df = evaluate_models(best_models, data_config)
# metrics_df

In [11]:
for model_name in ['logreg', 'xgb']:
    study_name=f'{model_name}_cp'
    target = 'completion'
    cp_model = get_best_model(study_name, storage, train_df, test_dfs, model_config, cp_features, target, mirror=MIRROR, noise_factor=NOISE_FACTOR, smote=SMOTE)
    study_name=f'{model_name}_fv'
    target = 'point_outcome'
    fv_model= get_best_model(study_name, storage, train_df, test_dfs, model_config, fv_features, target, mirror=MIRROR, noise_factor=NOISE_FACTOR, smote=SMOTE)


    etv_model = ETVModel(cp_model, fv_model)
    joblib.dump({'cp_model':cp_model, 'fv_model':fv_model, 'etv_model':etv_model, 'fv_features':fv_features, 'cp_features':cp_features}, f'../data/models/{model_name}_models.jblb')